In [1]:
import warnings
warnings.filterwarnings('ignore')



### 교차검증과 그리드 서치
- 머신러닝을 사용할때 모델의 정확도를 측정하기 위해 반드시 사용해야 하는 방법
- 딥러닝시에는 데이터의 크기가 크므로 이 방법은 사용할 필요가 없다.

In [2]:
import pandas as pd

wine = pd.read_csv('../data/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
# Feature, Target

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

### 검증 세트 추가
- 전 과정에서는 훈련세트와 테스트 세트만 가지고 작업을 하였지만 테스트 세트 작업의 결과로 파라미터를 조절하여 정확성을 높이면 실전(일반화)에는 아무런 의미가 없다.
- 이런 과정을 방지하기 위해 훈련세트, 검증세트, 테스트 세트로 구분하여 분석작업을 한다.


In [5]:
# 전체 세트중 훈련세트와 테스트 세트의 비율을 8:2로 분리한다.

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data,
    target, 
    test_size=0.2,
    random_state=42,
)

In [7]:
# 훈련세트중 훈련세트와 검증세트를 8:2의 기준으로 분리한다.
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input,
    train_target,
    test_size=0.2,
    random_state=42
)

In [8]:
# 확인
print('훈련세트 : ', sub_input.shape )
print('검증세트 : ', val_input.shape )
print('테스트세트 : ', test_input.shape )

훈련세트 :  (4157, 3)
검증세트 :  (1040, 3)
테스트세트 :  (1300, 3)


In [10]:
# 결정트리 모델

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print("train score :", dt.score(sub_input, sub_target))
print("valid score :", dt.score(val_input, val_target))

train score : 0.9971133028626413
valid score : 0.864423076923077


---
## 교차검증
- 교차검증의 한파트를 폴드라고 하며, 교차검증의 기본 Fold는 5이다. 
- 훈련세트와 검증세트를 바꾸어 가며 정확도를 구하는 방법이다.
- 전체의 대한 정확도는 해당 값들의 평균으로 구한다.


In [12]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00599575, 0.05247521, 0.00803375, 0.00705886, 0.00448704]),
 'score_time': array([0.00144529, 0.00061989, 0.00400519, 0.00054002, 0.00037789]),
 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [13]:
# 교차검증후 정확도 판단

import numpy as np

np.mean(scores['test_score'])

0.8554925223957948

### KFold를 이용한 방법 : 분할기를 사용한 교차 검증

In [14]:
from sklearn.model_selection import StratifiedKFold

In [15]:
splitter = StratifiedKFold(n_splits=5) # n_splits = 5가 기본
scores = cross_validate(dt, train_input, train_target, cv=splitter)

scores

{'fit_time': array([0.00932407, 0.00589275, 0.00635886, 0.00624609, 0.00469923]),
 'score_time': array([0.00086498, 0.00051618, 0.00072813, 0.000489  , 0.00036883]),
 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [16]:
np.mean(scores['test_score'])

0.8554925223957948

In [18]:
# KFold중 Fold를 10으로 나눠서 교차검증
splitter = StratifiedKFold(n_splits=10, random_state=42, shuffle=True) # n_splits = 5가 기본
scores = cross_validate(dt, train_input, train_target, cv=splitter)

scores

{'fit_time': array([0.01226997, 0.05691791, 0.00979185, 0.01158404, 0.00513506,
        0.00493407, 0.00499201, 0.00506806, 0.00493383, 0.004812  ]),
 'score_time': array([0.00089884, 0.00050402, 0.00059104, 0.00040078, 0.00027108,
        0.00026703, 0.000314  , 0.00027108, 0.00025606, 0.00027919]),
 'test_score': array([0.83461538, 0.88461538, 0.85384615, 0.85384615, 0.84615385,
        0.87307692, 0.86153846, 0.85549133, 0.85163776, 0.86705202])}

In [19]:
np.mean(scores['test_score'])

0.8581873425226026

---
### 그리드서치(GridSearch)를 이용한 최적의 Hyper Parameter 찾기

#### 결정트리의 Hyper Parameter값 찾기

In [23]:
# 일정한 값을 정한다. 
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [20]:
from sklearn.model_selection import GridSearchCV

In [26]:
gs = GridSearchCV(
    DecisionTreeClassifier(
        random_state=42,
        ),
    params,
    n_jobs= -1
)

In [28]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [32]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.9615162593804117
0.8653846153846154


In [33]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

---
### Random Search(랜덤 서치)
- 정해진 파라미터가 아니고 파라미터의 범위를 정해 최적의 값 찾기

In [34]:
# 확률 분포
from scipy.stats import uniform, randint



In [38]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
            'max_depth' :randint(20, 50),
            'min_samples_split' : randint(2, 25),
            'min_samples_leaf' : randint(1, 25)
        }

In [39]:
from sklearn.model_selection import RandomizedSearchCV


gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), 
                        params,
                        n_iter= 100,
                        n_jobs= -1,
                        random_state=42
                        )

In [40]:
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1501f3dd0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1501a13d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1502da570>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1501f0f50>},
                   random_state=42)

In [42]:
# 최적값

gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [44]:
dt = gs.best_estimator_

print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.8928227823744468
0.86
